In [1]:
# import libraries
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import scipy.stats as stats
import pandas as pd
import pingouin
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# import location and price df
df_location_price = pd.read_pickle('data/yelp_price_location.pkl')

In [3]:
df_location_price.head(2)

,categories,price,rating,review_count,Lat,Lon,state
0,Salad|Seafood|American (Traditional),2,3.5,372,38.997397,-77.026797,MD
1,Pizza|American (New)|Salad,2,3.5,192,38.919506,-77.224311,VA


In [4]:
# look at review count and rating vs state first split into 3 groups
# DC, VA, MD
DC = df_location_price[df_location_price.state == 'DC'][[
    'state', 'review_count', 'rating']].reset_index()
MD = df_location_price[df_location_price.state == 'MD'][[
    'state', 'review_count', 'rating']].reset_index()
VA = df_location_price[df_location_price.state == 'VA'][[
    'state', 'review_count', 'rating']].reset_index()

In [ ]:
# check for anova assumption:
# Normality:
# Caveat to this is, if group sizes are equal, the F-statistic is robust to violations of normality
# Homogeneity of variance
# Same caveat as above, if group sizes are equal, the F-statistic is robust to this violation
# Independent observations (assume independent most of the time)

In [5]:
# normality test 
stats.normaltest(DC.rating)
stats.normaltest(MD.rating)
stats.normaltest(VA.rating)
stats.normaltest(DC.review_count)
stats.normaltest(MD.review_count)
stats.normaltest(VA.review_count)
# not normal 

NormaltestResult(statistic=1457.8930410979126, pvalue=0.0)

In [7]:
# randomly generate 100 samples for 100000 times to generate normal distributions 
# for both review count and rating 
def boostrap_sample(samples, n):
    return np.random.choice(samples, size = n, replace = True )
# generate sample means
def sampling(samples, n, num):
    sample_means = []
    for i in range(num):
        sample_means.append(boostrap_sample(samples, n).mean())
    return sample_means
def sample_category(samples,n,num, feature):
    return sampling(samples[feature], n, num)

In [27]:
# create stacked dataframe for tukey_hsd and welch F test
def table_transform(datas, group_names, colname):
    '''
    datas: a list of data for comparision 
    group_names: a list of strings with group names, datas order should be same as group_names
    colname: string, the category to compare 
    return tukeyhsd table result and stacked table 
    '''
    df = pd.DataFrame()
    for i in range(len(group_names)):
        df[group_names[i]] = datas[i]
    stacked_df = df.stack().reset_index()
    stacked_df = stacked_df.rename(
        columns={'level_0': 'id', 'level_1': 'state', 0: colname})
    return stacked_df
    

In [41]:
# set up tukey hsd for post anova with significance
def tukey_hsd(stacked_df, colname):
    '''
    stacked_df: from table_transform, a stacked df 
    colname: string, the category to compare 
    return tukeyhsd table result and stacked table 
    '''
    MultiComp = MultiComparison(stacked_df[colname],
                                stacked_df['state'])
    return MultiComp.tukeyhsd().summary()

In [24]:
# create datasets for DC, MD, VA with review_count
# we knew it will be normal due to Central Limit Theorem 
DC_review = sample_category(DC,100,100000,'review_count')
MD_review = sample_category(MD,100,100000,'review_count')
VA_review = sample_category(VA,100,100000,'review_count')

In [29]:
stacked_df_review = table_transform([DC_review,MD_review,VA_review], ['DC','MD','VA'],colname='review_count')

In [35]:
# variance test
stats.levene(DC_review,MD_review,VA_review)
# variance is not equal 
# we will first use one way anova but be mindful of that due to same sample size
# we will also perform welch's F test 

LeveneResult(statistic=25515.202312377638, pvalue=0.0)

In [36]:
stats.f_oneway(DC_review, MD_review, VA_review)
# show significance with oneway anova 

F_onewayResult(statistic=353339.2944923518, pvalue=0.0)

In [42]:
# check with welch F test 
pingouin.welch_anova(dv='review_count', between='state',data=stacked_df_review)
# same result as oneway anova 

,Source,ddof1,ddof2,F,p-unc
0,state,2,177911.775,526408.072,0.0


In [43]:
# for more information using tukey_hsd
tukey_hsd(stacked_df_review,'review_count')
# MD has the most traffic then VA then DC

group1,group2,meandiff,p-adj,lower,upper,reject
DC,MD,-113.2925,0.001,-113.6134,-112.9717,True
DC,VA,-39.085,0.001,-39.4059,-38.7642,True
MD,VA,74.2075,0.001,73.8866,74.5284,True


In [44]:
# create datasets for DC, MD, VA with rating 
DC_rating = sample_category(DC,100,100000,'rating')
MD_rating = sample_category(MD,100,100000,'rating')
VA_rating = sample_category(VA,100,100000,'rating')

In [ ]:
stats.probplot(DC_rating, dist="norm", plot=plt)
plt.show()

In [ ]:
stats.probplot(MD_rating, dist="norm", plot=plt)
plt.show()

In [ ]:
stats.probplot(VA_rating, dist="norm", plot=plt)
plt.show()

In [45]:
# variance test
stats.levene(DC_rating,MD_rating,VA_rating)
# variance is not equal 
# we will first use one way anova but be mindful of that due to same sample size
# we will also perform welch's F test 

LeveneResult(statistic=881.5471868374034, pvalue=0.0)

In [46]:
stats.f_oneway(DC_rating, MD_rating,VA_rating)

F_onewayResult(statistic=320787.2190432029, pvalue=0.0)

In [47]:
# create stacked df for both tukey_hsd and welch F test
stacked_df_rating = table_transform([DC_rating, MD_rating, VA_rating], [
                                    'DC', 'MD', 'VA'], 'rating')

In [48]:
# check with welch F test 
pingouin.welch_anova(dv='rating', between='state',data=stacked_df_rating)
# same result as oneway anova 

,Source,ddof1,ddof2,F,p-unc
0,state,2,199397.179,297075.601,0.0


In [51]:
tukey_hsd(stacked_df_rating,'rating')
# MD has highest rating then VA then DC

group1,group2,meandiff,p-adj,lower,upper,reject
DC,MD,-0.289,0.001,-0.2899,-0.2881,True
DC,VA,-0.0621,0.001,-0.063,-0.0612,True
MD,VA,0.2269,0.001,0.226,0.2278,True


In [53]:
# check location correlation with most popular cuisines, ramen and bars 

In [52]:
# split the rows helper function
def split_rows(df, col1, col2, sep):
    '''
    input: 
    df: dataframe to use
    col1 and col2 (use col2 to split )
    return:
    a df table with col1 as common col and col2 split into multiple rows
    '''
    series = [pd.Series(row[col1], row[col2].split(sep))
              for _, row in df.iterrows()]
    table = pd.concat(series).reset_index()
    return table

In [60]:
table = split_rows(df_location_price,'Lat', 'categories', '|')
table = table.rename(columns = {'index': 'category', 0: 'Lat'})
# join table 
mergedtable = pd.merge(df_location_price,table, on = 'Lat')

In [61]:
# filter out only bars and ramen 
filtered_df = mergedtable[(mergedtable.category=='Bars')|(mergedtable.category =='Ramen')]
# make into a count table between state and category 
counts = filtered_df.groupby(['state','category']).size().reset_index()
# make into correct format for chisquare test 
chi_table= counts.pivot(index = 'state', columns='category', values=0)

In [68]:
chi_table

category,Bars,Ramen
state,,
DC,221,46
MD,61,8
VA,143,19


In [70]:
# sequence is DC, MD, VA
chi_array = np.array(chi_table)

In [71]:
chi_array

array([[221,  46],
       [ 61,   8],
       [143,  19]])

In [73]:
stats.chi2_contingency(chi_array)
# chi2 is 3.03
# p value is 0.218 > alpha (0.05)
# dof = 2 
# no depedency between location and ramen or bar 
# location doesn't seem to influence more ramen to be open in DC than MD and VA 

(3.039371664444074, 0.218780609974467, 2, array([[227.86144578,  39.13855422],
        [ 58.88554217,  10.11445783],
        [138.25301205,  23.74698795]]))